In [26]:
!conda install -y ipytest

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - ipytest


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipytest-0.9.1              |             py_0          14 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          14 KB

The following NEW packages will be INSTALLED:

  ipytest            conda-forge/noarch::ipytest-0.9.1-py_0



ipytest-0.9.1        | 14 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [27]:
import ipytest
ipytest.autoconfig()

In [20]:
from __future__ import print_function, absolute_import

#import findspark
#findspark.init()

from pyspark import SparkContext
from pyspark.sql import HiveContext
import pyspark.sql.functions as sf


def define_df(hive_context):
    "Dummy function"
    data = [('foo', 1),
            ('bar', 2)]
    schema = get_schema()
    df = hive_context.createDataFrame(data, schema=schema)
    return df


def filter_str(df, column, string):
    return df.filter(~(sf.regexp_extract(column, '({})'.format(string), 1) == string))


def get_schema():
    return ['name', 'age']

def main(sc, hc):
    df = define_df(hc).coalesce(1)
    filtered = filter_str(df, 'name', 'bar')
    

if __name__ == "__main__":
    sc = SparkContext.getOrCreate()
    hc = HiveContext(sc)
    main(sc, hc)

In [33]:
import shutil

import pytest
#import findspark
#findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext


APP_NAME = 'pytest-pyspark-tests'


@pytest.fixture(scope="session", autouse=True)
def spark_context(request):
    """Fixture to create the SparkContext.
    The tests run on real data, so it should run on the cluster
    where the actual data is present.
    Args:
        request: pytest.FixtureRequest object
    Returns:
        HiveContext for tests
    """
    conf = SparkConf().setAppName(APP_NAME)
    sc = SparkContext.getOrCreate(conf=conf)
    sc.setLogLevel("ERROR")
    # request.addfinalizer(lambda: sc.stop())
    yield sc
    sc.stop()


@pytest.fixture(scope="session", autouse=True)
def hive_context(spark_context, request):
    """
    Fixture to create the HiveContext.
    Args:
        spark_context: spark_context fixture
    Returns:
    HiveContext
    """
    try:
        shutil.rmtree('metastore_db')
    except:
        pass
    # request.addfinalizer(lambda: spark_context.stop())
    yield HiveContext(spark_context)

In [35]:
%%run_pytest

from __future__ import print_function, absolute_import
import pytest

#import findspark
#findspark.init()

import pyspark.sql.functions as sf  # more import if needed here



def test_define_df(spark_context, hive_context):
    df = define_df(hive_context).coalesce(1)
    row, = df.select(sf.max('age')).collect()
    assert row['max(age)'] == 2


def test_filter_str(hive_context):
    df = define_df(hive_context).coalesce(1)
    filtered = filter_str(df, 'name', 'bar')
    row, = filtered.collect()
    assert row['name'] == 'foo'


def test_get_schema():
    assert ['name', 'age'] == get_schema()

...                                                                                                                                         [100%]
================================================================ warnings summary =================================================================
tmp4ce73nbs.py::test_define_df
  /usr/local/spark/python/pyspark/sql/context.py:496: DeprecationWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
    warnings.warn(

tmp4ce73nbs.py::test_define_df
tmp4ce73nbs.py::test_define_df
  /usr/local/spark/python/pyspark/sql/context.py:75: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/warnings.html
3 passed, 3 warnings in 3.61s


In [ ]:
import logging
from pyspark.sql import SparkSession

def suppress_py4j_logging():
    logger = logging.getLogger(‘py4j’)
    logger.setLevel(logging.WARN)
    
def create_testing_pyspark_session():
    return (SparkSession.builder
    .master(‘local[2]’)
    .appName(‘my-local-testing-pyspark-context’)
    .enableHiveSupport()
    .getOrCreate())

In [ ]:
import unittest
import logging
from pyspark.sql import SparkSessionclass PySparkTest(unittest.TestCase):
 
@classmethod
def suppress_py4j_logging(cls):
    logger = logging.getLogger(‘py4j’)
    logger.setLevel(logging.WARN)
    
@classmethod
def create_testing_pyspark_session(cls):
    return (SparkSession.builder
    .master(‘local[2]’)
    .appName(‘my-local-testing-pyspark-context’)
    .enableHiveSupport()
    .getOrCreate())
 
@classmethod
def setUpClass(cls):
    cls.suppress_py4j_logging()
    cls.spark = cls.create_testing_pyspark_session()

@classmethod
def tearDownClass(cls):
    cls.spark.stop()

In [ ]:
def test_spark_context_fixture(spark_context):
    test_rdd = spark_context.parallelize([1, 2, 3, 4])

    assert test_rdd.count() == 4

In [11]:
import logging
import unittest

# here declare all the variables that are shared among unit test methods.
shared_variable = "foo"

# get the job module to be used given the job name
# for example if we want to tests on job_1
job_to_test = "job_1"


# here goes the class and the unit test methods

class SparkJobTests(unittest.TestCase):
    # methods below represent the test cases you'd want to run on a section of a job as described in the above code example
    def method_1(self):
        pass



# start the testing

if __name__ == '__main__':
    # run all tests main
    unittest.main()

    # only run a specific test lets say test contained in method_k
    suite = unittest.TestSuite()
    suite.addTest(Test("method_k"))
    runner = unittest.TextTestRunner()
    runner.run(suite)

E
ERROR: /home/jovyan/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/jovyan/'

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (errors=1)


SystemExit: True

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
%conda install pytest

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytest


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    iniconfig-1.0.1            |     pyh9f0ad1d_0           8 KB  conda-forge
    more-itertools-8.5.0       |             py_0          37 KB  conda-forge
    pluggy-0.13.1              |   py38h32f6830_2          29 KB  conda-forge
    py-1.9.0                   |     pyh9f0ad1d_0          74 KB  conda-forge
    pytest-6.0.2               |   py38h32f6830_0         419 KB  conda-forge
    toml-0.10.1                |     pyh9f0ad1d_0          18 KB  conda-forge
    ------------------------------------------------------------
                               